# Imports


In [1]:
import torch
import json
import re
import requests
import os
from sklearn.model_selection import train_test_split

In [125]:
# request_NER_TRAIN = requests.get("http://AdityaAhuja01.pythonanywhere.com/data/NLP_Data/NER_TRAIN_JUDGEMENT.json")
# request_NER_TEST = requests.get("http://AdityaAhuja01.pythonanywhere.com/data/NLP_Data/NER_TEST_JUDGEMENT.json")
# request_LR_VAL = requests.get("http://AdityaAhuja01.pythonanywhere.com/data/NLP_Data/Laptop_Review_Val.json")
# request_LR_TEST = requests.get("http://AdityaAhuja01.pythonanywhere.com/data/NLP_Data/Laptop_Review_Test.json")
# request_LR_TRAIN = requests.get("http://AdityaAhuja01.pythonanywhere.com/data/NLP_Data/Laptop_Review_Train.json")

# if not os.path.exists("./data"):
#     os.makedirs("./data")

# with open("./data/NER_train.json", "x") as file:
#     file.write(request_NER_TRAIN.text)

# with open("./data/NER_test.json", "x") as file:
#     file.write(request_NER_TEST.text)
    
# with open("./data/LR_Val.json", "x") as file:
#     file.write(request_LR_VAL.text)
    
# with open("./data/LR_test.json", "x") as file:
#     file.write(request_LR_TEST.text)

# with open("./data/LR_Train.json", "x") as file:
#     file.write(request_LR_TRAIN.text)

In [4]:
NER_train_file = open("./data/NER_train.json")
NER_train_json = json.load(NER_train_file)

NER_test_file = open("./data/NER_test.json")
NER_test_json = json.load(NER_test_file)

LR_train_file = open("./data/LR_Train.json")
LR_train_json = json.load(LR_train_file)

LR_val_file = open("./data/LR_Val.json")
LR_val_json = json.load(LR_val_file)

LR_test_file = open("./data/LR_Test.json")
LR_test_json = json.load(LR_test_file)

# Preprocessing the dataset

In [127]:
class LR_Preprocessor:
    def __init__(self,trainset,testset,valset):
        self.trainset = trainset
        self.testset = testset
        self.valset = valset

    def init_tags(self,dataset):
        labeled_outputs = []
        id = 0
        for entry in dataset:
            tag_entry = {}
            tag_entry["id"] = id 
            tag_entry["text"] = entry["raw_words"]
            tag_entry["labels"] = len(entry["words"])*["O"]
            for aspect in entry["aspects"]:
                for index in range(aspect["from"],aspect["to"]):
                    if index == aspect["from"]:
                        tag_entry["labels"][index] = "B"
                    else:
                        tag_entry["labels"][index] = "I"
                        
            labeled_outputs.append(tag_entry)   
            id += 1
        return labeled_outputs
        
    def initizalize(self):
        self.labeled_trainset = self.init_tags(self.trainset)
        self.labeled_valset = self.init_tags(self.valset)
        self.labeled_testset = self.init_tags(self.testset)
    
    def get_tagged_data(self):
        return self.labeled_trainset,self.labeled_valset,self.labeled_testset
        
            

In [128]:
def find_word_indices(test_str):
    word_indices = []
    start_index = 0
    for i in range(len(test_str)):

        if test_str[i] == " ":
            if start_index != i:
                word_indices.append((start_index, i - 1))
            start_index = i + 1

    if start_index != len(test_str):
        word_indices.append((start_index, len(test_str) - 1))
    return word_indices

In [129]:
def check_alphanumeric_in_word(word):
    for char in word:
        if char.isalnum():
            return True
    return False

In [130]:
class NER_Preprocessor:
    def __init__(self, trainset, testset, valset=None):
        self.trainset = trainset
        self.testset = testset
        self.valset = valset
        if valset is None:
            self.split_val()

    def split_val(self, split_train_false=False):
        if split_train_false:
            self.trainset = self.trainset + self.valset
        self.trainset, self.valset = train_test_split(
            self.trainset, test_size=0.15, random_state=42)

    def init_tags(self, dataset):
        labeled_output = []
        for entry in dataset:
            tag_entry = {}
            tag_entry["id"] = entry["id"]
            annotations = entry["annotations"][0]
            sentence = entry["data"]["text"].replace("\n", " ").replace("\t", " ")
            sentence = sentence
            tag_entry["text"] = ""
            tag_entry["labels"] = []
            no_tags = False
            if (len(annotations["result"]) == 0):
                no_tags = True
                continue
            
            word_ranges = find_word_indices(sentence)
            for i in range(len(word_ranges)):
                found = False
                for resultObj in annotations["result"]:
                    if (word_ranges[i][0] >= resultObj["value"]["start"] and word_ranges[i][1] <= resultObj["value"]["end"]):
                        tag_entry["text"] += sentence[word_ranges[i][0]:word_ranges[i][1]+1] + " "
                        tag_entry["labels"].append("I_" + resultObj["value"]["labels"][0])
                        found = True
                        break
                    
                if not found:
                    tag_entry["text"] += sentence[word_ranges[i][0]:word_ranges[i][1]+1] + " "
                    tag_entry["labels"].append("O")
                
            for i in range(len(tag_entry["labels"])):
                if i==0 and tag_entry["labels"][i] != "O":
                    tag_entry["labels"][i] = "B_" + tag_entry["labels"][i][2:]
                elif tag_entry["labels"][i] != "O" and tag_entry["labels"][i-1] == "O":
                    tag_entry["labels"][i] = "B_" + tag_entry["labels"][i][2:]
             
            labeled_output.append(tag_entry)
            
        for entry in range(len(labeled_output)):
            s = ""
            for wordnum in range(len(labeled_output[entry]["text"].split())):
                if not check_alphanumeric_in_word(labeled_output[entry]["text"].split()[wordnum]):
                    newword = labeled_output[entry]["text"].split()[wordnum]
                else:
                    newword = re.sub(r'[^\w\s]','',labeled_output[entry]["text"].split()[wordnum])
                s += newword + " "
            labeled_output[entry]["text"] = s
                
            
            
            
        return labeled_output

    def initialize(self):
        self.labeled_trainset = self.init_tags(self.trainset)
        self.labeled_valset = self.init_tags(self.valset)
        self.labeled_testset = self.init_tags(self.testset)

    def get_tagged(self):
        return (self.labeled_trainset, self.labeled_valset, self.labeled_testset)

In [131]:
NER_preprocessor = NER_Preprocessor(NER_train_json, NER_test_json)
NER_preprocessor.initialize()
NER_train,NER_val,NER_test = NER_preprocessor.get_tagged()

In [132]:
if (not os.path.exists("./data_processed")):
    os.makedirs("./data_processed")

with open("./data_processed/NER_train_tagged.json", "w") as file:
    json.dump(NER_train, file)
    
with open("./data_processed/NER_val_tagged.json", "w") as file:
    json.dump(NER_val, file)

with open("./data_processed/NER_test_tagged.json", "w") as file:
    json.dump(NER_test, file)

In [133]:
LR_Preprocessor = LR_Preprocessor(LR_train_json,LR_test_json,LR_val_json)
LR_Preprocessor.initizalize()
LR_train,LR_val,LR_test = LR_Preprocessor.get_tagged_data()

In [134]:
with open("./data_processed/LR_train_tagged.json", "w") as file:
    json.dump(LR_train, file)

with open("./data_processed/LR_val_tagged.json", "w") as file:
    json.dump(LR_val, file)
    
with open("./data_processed/LR_test_tagged.json", "w") as file:
    json.dump(LR_test, file)